# Classification
The task is to identify which category an object belongs to.

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

### Let's try to create an ML model for recognizing handwritten numbers. We download the data first.

In [ ]:
# Import MNIST data
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

### Now we can see what our task is. From the small greyscale pictures, decide what number is written on it. The correct answer is marked here as `target`. Since we know the correct answer, we can used some method based on the supervised learning.

In [ ]:
plt.figure(figsize=(10, 10))
for idx in range(25):
    plt.subplot(5, 5, idx+1)
    fig = plt.imshow(x_train[idx], cmap='gray')
    plt.title('Target {}'.format(y_train[idx]))
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)

### We will use the Neural Network. First we create a simple neural network a model, here using the Tensorflow library.

In [ ]:
model = tf.keras.models.Sequential([tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(256),
                                    tf.keras.layers.Dropout(0.25),
                                    tf.keras.layers.Dense(10, activation='softmax')
                                   ])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


### We now train the created model on the training data and then evaluate it on the test data.

In [ ]:
model.fit(x_train, y_train, epochs=20, batch_size=512, shuffle=True)

model.evaluate(x_test,  y_test, verbose=2)

### Now we can see the resulting classification, the caption above the image we are trying to classify is in the format {correct answer} - {prediction}

In [ ]:
plt.figure(figsize=(10, 10))

start = 100
for idx in range(25):
    plt.subplot(5,5,idx+1)
    fig = plt.imshow(x_test[idx+start], cmap='gray')
#     print(model.predict(x_test[idx+start].reshape(1, 28, 28)), np.argmax(model.predict(x_test[idx+start].reshape(1, 28, 28))))
    plt.title('{} - {}'.format(y_test[idx+start], np.argmax(model.predict(x_test[idx+start].reshape(1, 28, 28)))))
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)

### In the figure below we can isnpect a few incorrect predictions

In [ ]:
ypredict = np.argmax(model.predict(x_test), axis=1)
wrong = np.where(ypredict-y_test)[0]
plt.figure(figsize=(10, 10))


start = 100
for idx in range(25):
    plt.subplot(5,5,idx+1)
    fig = plt.imshow(x_test[wrong[idx]], cmap='gray')
    plt.title('{} - {}'.format(y_test[wrong[idx]], ypredict[wrong[idx]]))
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
